In [7]:
pip install sklearn azureml.train

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 117.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 92.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: azureml-core
    Found existing installation: azureml-core 1.47.0
    Uninstalling azureml-core-1.47.0:
      Successfully uninstalled azureml-core-1.47.0
Note: you may need to restart the kernel to use updated packages.


In [26]:
%%writefile hp_train.py
from sklearn.svm import SVC
import argparse
import numpy as np
from sklearn.metrics import mean_squared_error
import joblib
from sklearn.model_selection import train_test_split
import pandas as pd
from azureml.core.run import Run
from azureml.data.dataset_factory import TabularDatasetFactory

def clean_data(data):
    input_cols = np.arange(1,14,1)
    output_cols = [14]
    x_df = data[data.columns[input_cols]]
    y_df = data[data.columns[output_cols]]
    return x_df, y_df

def main():
    # Add arguments to script
    parser = argparse.ArgumentParser()

    parser.add_argument('--C', type=float, default=1.0, help="Regularization Parameter")
    parser.add_argument('--kernel', type=str, default='rbf', help="Kernel Type")
    parser.add_argument('--degree', type=int, default=3, help="Degree of polynomial function, only applicable if kernel='poly'")
    parser.add_argument('--gamma', type=float, default='0.1', help="Kernel Coefficient")

    args = parser.parse_args()

    run = Run.get_context()

    run.log("Regularization Parameter:", np.float(args.C))
    run.log("Kernel:", str(args.kernel))
    run.log("Degree:", np.int(args.degree))
    run.log("Gamma:", np.float(args.gamma))

    model = SVC(C=args.C, kernel=args.kernel, degree=args.degree, gamma=args.gamma).fit(x_train, y_train)

    wAUC = model.roc_auc_score(x_test, y_test, average='weighted')
    run.log("weighted AUC", np.float(wAUC))

if __name__ == '__main__':
    #Had to relocate my user code here otherwise Job would not start

    ds = pd.read_csv('salary_cleaned.csv')
    
    x, y = clean_data(ds)

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=101)

    main()

Writing hp_train.py


In [27]:
import argparse
parser = argparse.ArgumentParser()

parser.add_argument('--C', type=float, default=1.0, help="Regularization Parameter")
parser.add_argument('--kernel', type=str, default='rbf', help="Kernel Type")
parser.add_argument('--degree', type=int, default=3, help="Degree of polynomial function")

args = parser.parse_args()

_StoreAction(option_strings=['--C'], dest='C', nargs=None, const=None, default=1.0, type=<class 'float'>, choices=None, required=False, help='Regularization Parameter', metavar=None)

_StoreAction(option_strings=['--kernel'], dest='kernel', nargs=None, const=None, default='rbf', type=<class 'str'>, choices=None, required=False, help='Kernel Type', metavar=None)

_StoreAction(option_strings=['--degree'], dest='degree', nargs=None, const=None, default=3, type=<class 'int'>, choices=None, required=False, help='Degree of polynomial function', metavar=None)

usage: ipykernel_launcher.py [-h] [--C C] [--kernel KERNEL] [--degree DEGREE]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/azureuser/.local/share/jupyter/runtime/kernel-fa4155c5-710c-406e-aac4-d72324ef9b49.json


SystemExit: 2